In [ ]:
# pylint: disable=line-too-long,useless-suppression
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""

DESCRIPTION:
    This sample demonstrates how to use Agent operations with the Microsoft Fabric grounding tool from
    the Azure Agents service using a synchronous client.

USAGE:
  

    Before running the sample:

    Set this environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) FABRIC_CONNECTION_ID  - The ID of the Fabric connection, in the format of:
       /subscriptions/{subscription-id}/resourceGroups/{resource-group-name}/providers/Microsoft.MachineLearningServices/workspaces/{workspace-name}/connections/{connection-name}
       
       GO here to create a connection:
        https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/fabric?pivots=portal
"""

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FabricTool, ListSortOrder
from dotenv import load_dotenv

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")
FABRIC_CONNECTION_NAME = os.getenv("FABRIC_CONNECTION_NAME")


project_client = AIProjectClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(),
)

connection_id = project_client.connections.get(FABRIC_CONNECTION_NAME).id
print(connection_id)
# Initialize an Agent Fabric tool and add the connection id
fabric = FabricTool(connection_id=connection_id)

# Create an Agent with the Fabric tool and process an Agent run
agents_client = project_client.agents

agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="my-fabric-data-agent",
    instructions="You are a helpful agent",
    tools=fabric.definitions,
)
# [END create_agent_with_fabric_tool]
print(f"Created Agent, ID: {agent.id}")

# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")


/subscriptions/b8b550c2-b222-4bfe-a36f-5e4b306ecd5c/resourceGroups/fabricdemos/providers/Microsoft.CognitiveServices/accounts/fabric-demos-ai-projnew-resource/projects/fabric-demos-ai-projnew/connections/fabric-data-agent
Created Agent, ID: asst_xiu3OAahzhxvX7tu6jSlKLEN
Created thread, ID: thread_tZku5ojW3WRUlAYq4JUi4Eoy


In [5]:
# Create thread for communication
def call_agent(agents_client, thread, prompt):
    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt,
    )
    print(f"Created message, ID: {message.id}")

    # Create and process an Agent run in thread with tools
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the Agent when done
    # agents_client.delete_agent(agent.id)
    # print("Deleted agent")

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role}: {last_text.text.value}")

In [6]:
prompt = "How many passengers are the most popular in trips?"
call_agent(agents_client, thread, prompt)

Created message, ID: msg_oRHQ1gJb3nYIMzkHpTUcPr8p
Run finished with status: completed
user: How many passengers are the most popular in trips?
assistant: The most popular number of passengers per trip in ride-sharing services is 1 passenger【3:0†source】.
